In [1]:
from datasets_dataloader_pytorch import CustomDataset, load_data
from sklearn.model_selection import train_test_split
import numpy as np
import torch

from CAMELOT import CamelotModel

In [2]:
dataset = CustomDataset()

# Stratified Sampling for train and val
train_idx, test_idx = train_test_split(np.arange(len(dataset)),
                                             test_size=0.4,
                                             random_state=2323,
                                             shuffle=True,
                                             stratify=np.argmax(dataset.y,axis=-1))

# Subset dataset for train and val
train_dataset = dataset.get_subset(train_idx)
test_dataset = dataset.get_subset(test_idx)

train_loader, val_loader, test_loader = load_data(train_dataset, test_dataset, test_dataset)

# train_idx, val_idx = train_test_split(np.arange(len(train_idx)),
#                                              test_size=0.5,
#                                              random_state=2323,
#                                              shuffle=True,
#                                              stratify=np.argmax(train_dataset.y,axis=-1))

# train_dataset = dataset.get_subset(train_idx)
# val_dataset = dataset.get_subset(val_idx)

# train_loader, val_loader, test_loader = load_data(train_dataset, val_dataset, test_dataset)


MIMIC data has been subsettted to the following features: 
 ['DBP', 'HR', 'RR', 'SBP', 'SPO2', 'TEMP'].


100%|██████████| 7701/7701 [00:23<00:00, 331.90it/s]


In [3]:
model = CamelotModel(input_shape=(6, 6))

In [4]:
model.initialize((torch.tensor(train_dataset.x), torch.tensor(train_dataset.y)), (torch.tensor(test_dataset.x), torch.tensor(test_dataset.y)))

torch.Size([64, 6, 6, 32]) torch.Size([64, 6, 32, 6])
torch.Size([64, 6, 6, 32]) torch.Size([64, 6, 32, 1])
torch.Size([64, 6, 6, 1]) torch.Size([64, 6, 6, 32])
torch.Size([64, 6, 32]) torch.Size([1, 6, 1])
tensor([[ 0.0273,  0.0151, -0.0017,  ...,  0.0080,  0.0183, -0.0305],
        [ 0.0385,  0.0121, -0.0044,  ...,  0.0173,  0.0263, -0.0265],
        [ 0.0225,  0.0067,  0.0032,  ...,  0.0094,  0.0223, -0.0305],
        ...,
        [ 0.0236,  0.0098, -0.0012,  ...,  0.0132,  0.0184, -0.0300],
        [ 0.0262,  0.0040, -0.0045,  ...,  0.0206,  0.0330, -0.0248],
        [ 0.0230,  0.0054, -0.0013,  ...,  0.0117,  0.0206, -0.0291]],
       grad_fn=<SumBackward1>)


AttributeError: 'CamelotModel' object has no attribute 'weights'